In [47]:
#Esta é uma versão stand alone, precisa do mongoDB instalado local.
import pandas as pd # org orra t
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import datetime as dt
from datetime import timedelta
from pandas.tseries.offsets import BDay
import pymongo as md
from pymongo import MongoClient
from bson.objectid import ObjectId
import plotly.graph_objects as go 
import plotly.express as px
from scipy.stats import norm
#import yfinance as yf
#yf.pdr_override()

In [18]:
#busca ativos do Índice Ibovespa
c = 0
p = 18
acoesb3 = pd.DataFrame(columns=["empresa","ticker","indice"])
while c<p:
    page = rq.get('http://bvmf.bmfbovespa.com.br/indices/AcoesIndice.aspx?idioma=pt-BR&pg='+str(c))# somente as do indice
    soup = bs(page.content, 'html.parser')
    table = soup.find('div', class_='large-12 columns')
    lista = table.find_all('td')
    i = 0
    while i < len(lista)-3:
        acoesb3 = acoesb3.append({"empresa": ' '.join((lista[i].text.split())),
                                  "ticker": ''.join(lista[i+1].text.split()),
                                  "indice": ' '.join(lista[i+2].text.split())}, ignore_index=True)
        i+=3
    c+=1
#retirando fundos de investimento   
acoes = acoesb3.query("indice != 'IFIX'")
#incluindo índice Ibovespa
acoes = acoes.append({"empresa":'IBOVESPA', "ticker":'^BVSP', "indice":'IBOV'}, ignore_index=True)
acoes.set_index('ticker', inplace = True)
#removendo ação que não possuí preço
acoes = acoes.drop('K1EY34')
acoes = acoes.drop_duplicates()
acoes.reset_index(inplace=True)

In [76]:

#conectando ao mongo
con = MongoClient()
db = con.price

#consultando precos
c=0
failed =[]
for i in acoes['ticker']: 
    #verifica se há na base preços para o ativo
    if i in db.list_collection_names():
        #parametriza consulta diária se houver dados para ação
        lastd = db.get_collection(i).find_one(sort=[("Date", -1)])["Date"]
        if lastd == dt.date.today():
            continue
        if lastd.weekday() in (4,5,6):
            continue        
        else:
            try:    
                if i =='^BVSP':
                    price = pdr.get_data_yahoo(i,start=lastd, end=dt.date.today())
                else:
                    price = pdr.get_data_yahoo(i+'.SA',start=lastd, end=dt.date.today())
            except KeyError:
                failed.append(i)
                pass
    #parametriza consulta histórica quando não há dados
    else:
        lastd=dt.datetime(2010, 1, 4)
        #parametriza consulta histórica
        try:
            if i =='^BVSP':
                price = pdr.get_data_yahoo(i,start=lastd, end=dt.date.today())
            #Ações
            else:
                price = pdr.get_data_yahoo(i+'.SA',start=lastd, end=dt.date.today())
        except KeyError:
            failed.append(i)
            pass
            
    coll = db[i]
    price = price.reset_index()
    price = price.dropna()
    data_dict = price.to_dict("records")
    coll.insert_many(data_dict)
    n = coll.count()
    c+=1
    #print(i, n,c)
print("Ativos atualizados:", len(list(db.list_collection_names())))
print("Ativos não atualizados:", failed)

Ativos atualizados: 521
Ativos não atualizados: []


In [135]:
#Análise Descritiva 
#Medidas de Tendência Central
def medida_central(col):
    #+print("Medidas de Tendência Central") 
    #print(Nomes[i])
    print('Média:', round(col.mean(),2 ))
    print('Mediana:',round(col.median(),2))
    print('Moda:', round(col.mode().iloc[0]),2)
    print('Ponto Médio:',round((max(col) - min(col))/2,2))
    print("\n")   
    
def medida_dispersao(col):
    #print("Medidas de Dispersão")
    #print(Nomes[i])
    print('Amplitude:',(round(max(col) - min(col),2)))
    d1 = round(col.std(),2)
    print('Desvio Padrão σ:',d1)
    print('Variância:', round(col.var(),2))
    print('Coeficiente de variação', round(col.std()/col.median(),2))
    print('Dispersão em relação aos Desvios Padrão')
    print('Itens a 1 σ ', round(col[col[col >((col.mean()+d1)*(-1))]<((col.mean()+d1))].count()/col.count()*100,2),'%')
    print('Itens a 2 σ ', round(col[col[col >((col.mean()+d1)*(-2))]<((col.mean()+d1)*2)].count()/col.count()*100,2),'%')
    print('Itens a 3 σ ', round(col[col[col >((col.mean()+d1)*(-3))]<((col.mean()+d1)*3)].count()/col.count()*100,2),'%')
    print('\n')  

def distr_freq(col,classe):
    freq = pd.DataFrame(columns=['Classe','Limite_Inf','Pto_Médio','Limite_Sup','Fronteira','Freq_Abs','Freq_Relat',
          'Freq_Acum.','Freq_Acum_%'])
    inter = (max(col) - min(col))//classe
    tot = col.count()
    menor = int(min(col)//1)
    #lista com intervalo da frequencia
    c=ac=ct= 0
    if inter == 0.0:
        return ("Não Disponível")
    else:
        while menor < max(col):
                li = menor 
                ls = menor+inter 
                for i in col:
                    if  li-0.5 < i <= ls-0.5:
                        ct = ct + 1
                ac = ac + ct
                freq = freq.append({"Classe":c, "Limite_Inf":li, "Pto_Médio":((li+ls)/2),"Limite_Sup":ls, 
                                    "Fronteira":(ls-1+0.5), "Freq_Abs":ac, "Freq_Relat" :round(ac/tot,2),
                                    "Freq_Acum.":ac, "Freq_Acum_%":round(ac/tot,2)}, ignore_index=True)
                c = c + 1
                menor = menor + inter
        return (freq)

def medidas_associacao(col,col2,i,i2):
    #print("Medidas de Associação")
    print("A correlação  entre as ações")
    print( i.replace(".SA",""), "/", i2.replace(".SA",""), "é",round(col.corr(col2),2))  #np.corrcoef
    print("\nE a Covariância")
    print(np.cov(col,col2) )
    print("\n") 

In [ ]:
#get_collection("PETR4").estimated_document_count()
# db.PETR4.count_documents({})

In [138]:

for i in acoes['ticker']:
    docs = db.get_collection(i).find({},{"_id": 0, "Low":0, "High":0, "Open":0, "Volume":0, "Adj Close":0})
    bp = pd.DataFrame(list(docs))
    print("Medidas de Tendências Central", i )
    medida_central(bp.Close)
    print("Medidas de Dispersão", i )
    medida_dispersao(bp.Close)
    print("Medidas de Distribuição de Frequência", i )
    print(distr_freq(bp.Close,5))

    #fig = go.Figure( )
    #fig.add_trace(go.Scatter(x=bp.Date, y=bp.Close,mode='lines'))
    #fig.update_layout(#barmode='stack',
    #title="Preços Ações na decada 2010-2020",#xaxis_title="2010-2020",
    #margin=dict(l=20, r=20, t=50, b=10), #yaxis_title="Preço",
    #font=dict(family="Arial, monospace", size=18, color="#7f7f7f"
    #    )
    #)
    #fig.show()

Medidas de Tendências Central MMMC34
Média: 132.68
Mediana: 145.9
Moda: 46.0 2
Ponto Médio: 91.29


Medidas de Dispersão MMMC34
Amplitude: 182.59
Desvio Padrão σ: 51.36
Variância: 2638.31
Coeficiente de variação 0.35
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  81.78 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência MMMC34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0        41.0       59.0        77.0       76.5     405.0        0.20   
1     1.0        77.0       95.0       113.0      112.5    1074.0        0.53   
2     2.0       113.0      131.0       149.0      148.5    2161.0        1.08   
3     3.0       149.0      167.0       185.0      184.5    3805.0        1.89   
4     4.0       185.0      203.0       221.0      220.5    5806.0        2.89   
5     5.0       221.0      239.0       257.0      256.5    7815.0        3.89   

   Freq_Acum.  Freq_Acum_%  
0       405.0         0.20  
1      

Medidas de Tendências Central AIGB34
Média: 194.03
Mediana: 200.04
Moda: 211.0 2
Ponto Médio: 73.6


Medidas de Dispersão AIGB34
Amplitude: 147.2
Desvio Padrão σ: 24.13
Variância: 582.21
Coeficiente de variação 0.12
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  92.3 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência AIGB34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0        93.0      107.5       122.0      121.5       8.0        0.01   
1     1.0       122.0      136.5       151.0      150.5      78.0        0.08   
2     2.0       151.0      165.5       180.0      179.5     277.0        0.30   
3     3.0       180.0      194.5       209.0      208.5     891.0        0.97   
4     4.0       209.0      223.5       238.0      237.5    1811.0        1.96   
5     5.0       238.0      252.5       267.0      266.5    2733.0        2.96   

   Freq_Acum.  Freq_Acum_%  
0         8.0         0.01  
1        

Medidas de Tendências Central AMZO34
Média: 1623.31
Mediana: 1221.57
Moda: 355.0 2
Ponto Médio: 3633.07


Medidas de Dispersão AMZO34
Amplitude: 7266.15
Desvio Padrão σ: 1524.54
Variância: 2324232.07
Coeficiente de variação 1.25
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  78.95 %
Itens a 2 σ  97.84 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência AMZO34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0       153.0      879.5      1606.0     1605.5    1385.0        0.67   
1     1.0      1606.0     2332.5      3059.0     3058.5    3004.0        1.44   
2     2.0      3059.0     3785.5      4512.0     4511.5    5006.0        2.41   
3     3.0      4512.0     5238.5      5965.0     5964.5    7037.0        3.38   
4     4.0      5965.0     6691.5      7418.0     7417.5    9117.0        4.38   
5     5.0      7418.0     8144.5      8871.0     8870.5   11198.0        5.38   

   Freq_Acum.  Freq_Acum_%  
0      1385.0         0.

Desvio Padrão σ: 27.08
Variância: 733.28
Coeficiente de variação 0.15
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  89.54 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência APTV34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0        93.0      105.0       117.0      116.5      13.0        0.05   
1     1.0       117.0      129.0       141.0      140.5      35.0        0.15   
2     2.0       141.0      153.0       165.0      164.5     123.0        0.51   
3     3.0       165.0      177.0       189.0      188.5     267.0        1.12   
4     4.0       189.0      201.0       213.0      212.5     503.0        2.10   
5     5.0       213.0      225.0       237.0      236.5     742.0        3.10   

   Freq_Acum.  Freq_Acum_%  
0        13.0         0.05  
1        35.0         0.15  
2       123.0         0.51  
3       267.0         1.12  
4       503.0         2.10  
5       742.0         3.10  
Medidas d

Medidas de Tendências Central BAHI3
Média: 66.1
Mediana: 59.37
Moda: 60.0 2
Ponto Médio: 60.44


Medidas de Dispersão BAHI3
Amplitude: 120.88
Desvio Padrão σ: 32.33
Variância: 1044.95
Coeficiente de variação 0.54
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  78.75 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência BAHI3
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0        20.0       32.0        44.0       43.5     632.0        0.25   
1     1.0        44.0       56.0        68.0       67.5    2274.0        0.90   
2     2.0        68.0       80.0        92.0       91.5    4194.0        1.65   
3     3.0        92.0      104.0       116.0      115.5    6432.0        2.54   
4     4.0       116.0      128.0       140.0      139.5    8966.0        3.53   
5     5.0       140.0      152.0       164.0      163.5   11503.0        4.53   

   Freq_Acum.  Freq_Acum_%  
0       632.0         0.25  
1      2274.

...

Medidas de Tendências Central TPIS3
Média: 5.69
Mediana: 4.88
Moda: 2.0 2
Ponto Médio: 6.29


Medidas de Dispersão TPIS3
Amplitude: 12.58
Desvio Padrão σ: 3.31
Variância: 10.94
Coeficiente de variação 0.68
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  76.99 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência TPIS3
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0         0.0        1.0         2.0        1.5     190.0        0.07   
1     1.0         2.0        3.0         4.0        3.5     984.0        0.38   
2     2.0         4.0        5.0         6.0        5.5    2446.0        0.94   
3     3.0         6.0        7.0         8.0        7.5    4132.0        1.59   
4     4.0         8.0        9.0        10.0        9.5    6303.0        2.43   
5     5.0        10.0       11.0        12.0       11.5    8794.0        3.39   
6     6.0        12.0       13.0        14.0       13.5   11388.0        4.39 

5      5804.0         4.79  
Medidas de Tendências Central UNIP6
Média: 10.44
Mediana: 4.56
Moda: 4.0 2
Ponto Médio: 20.36


Medidas de Dispersão UNIP6
Amplitude: 40.72
Desvio Padrão σ: 11.25
Variância: 126.65
Coeficiente de variação 2.47
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  78.3 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência UNIP6
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0         1.0        5.0         9.0        8.5    1878.0        0.72   
1     1.0         9.0       13.0        17.0       16.5    3875.0        1.49   
2     2.0        17.0       21.0        25.0       24.5    5997.0        2.31   
3     3.0        25.0       29.0        33.0       32.5    8360.0        3.22   
4     4.0        33.0       37.0        41.0       40.5   10949.0        4.22   
5     5.0        41.0       45.0        49.0       48.5   13543.0        5.22   

   Freq_Acum.  Freq_Acum_%  
0      1878.0   

Moda: 18.0 2
Ponto Médio: 20.3


Medidas de Dispersão VLID3
Amplitude: 40.6
Desvio Padrão σ: 9.29
Variância: 86.34
Coeficiente de variação 0.45
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  79.12 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência VLID3
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0         6.0       10.0        14.0       13.5     206.0        0.08   
1     1.0        14.0       18.0        22.0       21.5    1535.0        0.59   
2     2.0        22.0       26.0        30.0       29.5    3312.0        1.28   
3     3.0        30.0       34.0        38.0       37.5    5662.0        2.19   
4     4.0        38.0       42.0        46.0       45.5    8241.0        3.18   
5     5.0        46.0       50.0        54.0       53.5   10832.0        4.18   

   Freq_Acum.  Freq_Acum_%  
0       206.0         0.08  
1      1535.0         0.59  
2      3312.0         1.28  
3      5662.0         2

5     10070.0         4.97  
Medidas de Tendências Central WGBA34
Média: 111.7
Mediana: 112.08
Moda: 105.0 2
Ponto Médio: 16.5


Medidas de Dispersão WGBA34
Amplitude: 33.0
Desvio Padrão σ: 7.87
Variância: 61.94
Coeficiente de variação 0.07
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  83.46 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência WGBA34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0        99.0      102.0       105.0      104.5      70.0        0.26   
1     1.0       105.0      108.0       111.0      110.5     194.0        0.71   
2     2.0       111.0      114.0       117.0      116.5     408.0        1.50   
3     3.0       117.0      120.0       123.0      122.5     649.0        2.39   
4     4.0       123.0      126.0       129.0      128.5     920.0        3.38   
5     5.0       129.0      132.0       135.0      134.5    1192.0        4.38   

   Freq_Acum.  Freq_Acum_%  
0        70.

Medidas de Tendências Central XRXB34
Média: 158.95
Mediana: 155.91
Moda: 154.0 2
Ponto Médio: 17.52


Medidas de Dispersão XRXB34
Amplitude: 35.03
Desvio Padrão σ: 9.25
Variância: 85.52
Coeficiente de variação 0.06
Dispersão em relação aos Desvios Padrão
Itens a 1 σ  74.19 %
Itens a 2 σ  100.0 %
Itens a 3 σ  100.0 %


Medidas de Distribuição de Frequência XRXB34
   Classe  Limite_Inf  Pto_Médio  Limite_Sup  Fronteira  Freq_Abs  Freq_Relat  \
0     0.0       142.0      145.5       149.0      148.5       7.0        0.08   
1     1.0       149.0      152.5       156.0      155.5      52.0        0.56   
2     2.0       156.0      159.5       163.0      162.5     119.0        1.28   
3     3.0       163.0      166.5       170.0      169.5     191.0        2.05   
4     4.0       170.0      173.5       177.0      176.5     282.0        3.03   
5     5.0       177.0      180.5       184.0      183.5     375.0        4.03   

   Freq_Acum.  Freq_Acum_%  
0         7.0         0.08  
1        